In [ ]:
%matplotlib inline
import numpy as np
import os
import tifffile as tif
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import pickle

inc_array = pickle.load( open( f'data/incorrect.p', "rb" ) )
line_width = 2


li = os.listdir(f'data/combined')
name_list = []
for item in li:
    if item[-4:] == 'tiff':
        name_list.append(item)

nl = []
num_l = []
for i, name in zip(range(len(name_list)), name_list):
    if inc_array[i] == 1:
        nl.append(name)
        num_l.append(i)

im_num = 0

def adj_input():
    o = input('Remove image from dataset? (y) to remove, press "Enter" othewise')
    if o != 'y':
        o = 'n'
    

        n = input('Number of rings? if already correct, press "Enter"')
        if n == '':
            n = 0
        r = input('Reset rings? (y) if reset, press "Enter" otherwise')
        if r != 'y':
            r = 'n'
    else:
        r= 'n'
        n = 0

    return r, int(n), o

# Repeat following cells until finished
<a id='start_cell'></a>

In [ ]:
pex = tif.imread(f'data/combined/{nl[im_num]}')[0]
circles, radii = pickle.load(open( f'data/analysis/{nl[im_num][:-5]}/circle_data.p', "rb" ) )
                             
%matplotlib inline 
fig, ax = plt.subplots(figsize=(10,10))
plt.title(nl[im_num][:-5], fontsize = 20)
ax.imshow(pex, cmap='Reds')
if type(circles[0]) == np.float64:
    x,y = circles
    rad = radii
    circ = Circle((x,  y), rad - line_width, color='black', fill = False, alpha = 0.5)
    ax.add_patch(circ)
    circ = Circle((x,  y), rad + line_width, color='black', fill = False, alpha = 0.5)
    ax.add_patch(circ)
    ax.plot(x,y, 'k.')
    ax.plot([x, x+rad],[y,y], 'k')
else:
    for circle, rad in zip(circles, radii):
        x,y = circle

        circ = Circle((x,  y), rad - line_width, color='black', fill = False, alpha = 0.5)
        ax.add_patch(circ)
        circ = Circle((x,  y), rad + line_width, color='black', fill = False, alpha = 0.5)
        ax.add_patch(circ)
        ax.plot(x,y, 'k.')
        ax.plot([x, x+rad],[y,y], 'k')

In [ ]:
reset_rings, nrings, remove_img = adj_input()

In [ ]:
%matplotlib notebook

if remove_img == 'y':
    clist = None
    rads = None
else:    

    if reset_rings == 'n':
        clist = circles
        rads = radii

    else:
        rads = []
        def onclick(event, l):


            x, y = event.xdata, event.ydata

            if event.button == 3:

                if len(l)%2 == 0:
                    ax.lines[-1].remove()
                    ax.lines[-1].remove()
                    ax.patches[-1].remove()
                    ax.patches[-1].remove()
                    x2, y2 = l[-1]
                    l.pop(-1)
                    ax.lines[-1].remove()
                    x1, y1 = l[-1]
                    l.pop(-1)

                    d1 = np.linalg.norm([x-x1, y-y1])
                    d2 = np.linalg.norm([x-x2, y-y2])

                    if d1 < d2:

                        xc, yc = x,y
                        xo, yo = x2, y2        

                    else:

                        xc, yc = x1,y1
                        xo, yo = x, y

                    l.append((xc,yc))
                    ax.plot(xc,yc, 'k.')

                    l.append((xo,yo))
                    ax.plot(xo,yo, 'k.')
                    ax.plot([xo, l[-2][0]],[yo, l[-2][1]], 'k')
                    rad = np.sqrt((xo - l[-2][0])**2 + (yo - l[-2][1])**2 )
                    circ = Circle((l[-2][0],  l[-2][1]), rad - line_width, color='black', fill = False, alpha = 0.5)
                    ax.add_patch(circ)
                    circ = Circle((l[-2][0],  l[-2][1]), rad + line_width, color='black', fill = False, alpha = 0.5)
                    ax.add_patch(circ)

                else:
                    ax.lines[-1].remove()
                    l.pop(-1)
            else:

                l.append((x, y))
                if len(l)%2 == 0:
                    ax.plot(x,y, 'k.')
                    ax.plot([x, l[-2][0]],[y, l[-2][1]], 'k')
                    rad = np.sqrt((x - l[-2][0])**2 + (y - l[-2][1])**2 )

                    circ = Circle((l[-2][0],  l[-2][1]), rad - line_width, color='black', fill = False, alpha = 0.5)
                    ax.add_patch(circ)
                    circ = Circle((l[-2][0],  l[-2][1]), rad + line_width, color='black', fill = False, alpha = 0.5)
                    ax.add_patch(circ)


                else:
                    ax.plot(x,y, 'k.')

        clist = []
        cnt = 0
        fig, ax = plt.subplots(figsize=(10,10))
        plt.title(nl[im_num][:-5], fontsize = 20)
        ax.imshow(pex, cmap='Reds')


        circ = 0

        cid = fig.canvas.mpl_connect('button_press_event', lambda evt: onclick(evt, clist))

In [ ]:
pth = f'data/analysis/{nl[im_num][:-5]}'


if reset_rings == 'y':
    rads = []
    
    plt.savefig(f'{pth}/updated_circles.png')
    for i in range(int(len(clist)/2)):
        rads.append(np.linalg.norm([clist[i*2][0]-clist[i*2+1][0], clist[i*2][1]-clist[i*2+1][1]]))

    centers = clist[::2]
    
else:
    centers = clist

if remove_img == 'n':
    if nrings> len(clist):

        for i in range(nrings-len(centers)):
            centers.append(None)
            rads.append(None)

pickle.dump( [centers, rads], open( f'{pth}/circle_data_manual_update.p', "wb" ) )

im_num +=1

if im_num == len(nl):
    print("FINISHED")

[Back to top](#start_cell)